# Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [4]:
from secrets import IEX_CLOUD_API_TOKEN

#### Sample of aapl stock get request

In [45]:
ticker = 'AAPL'
endpoint = 'https://sandbox.iexapis.com/stable'
api_url = endpoint + f'/stock/{ticker}/quote/?token={IEX_CLOUD_API_TOKEN}'

In [6]:
# get data
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SDAAQN', 'calculationPrice': 'close', 'open': 0, 'openTime': None, 'openSource': 'ficlaofi', 'close': 0, 'closeTime': None, 'closeSource': 'lfciofia', 'high': 0, 'highTime': None, 'highSource': None, 'low': 0, 'lowTime': None, 'lowSource': None, 'latestPrice': 147.21, 'latestSource': 'Close', 'latestTime': 'August 11, 2021', 'latestUpdate': 1677978809559, 'latestVolume': None, 'iexRealtimePrice': 151.97, 'iexRealtimeSize': 61, 'iexLastUpdated': 1646724708474, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 148.8, 'previousVolume': 70607822, 'change': 0.27, 'changePercent': 0.00181, 'volume': None, 'iexMarketPercent': 91.8101669217797, 'iexVolume': 768361, 'avgTotalVolume': 81602076, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexAskPrice': 0, 'iexAs

# Data Prep

In [7]:
columns_ = ['ticker', 'stock_price', 'market_cap', 'no_of_shares_tobuy']

df = pd.DataFrame(columns=columns_)

In [9]:
for ticker in stocks['Ticker']:
    api_url = endpoint + f'/stock/{ticker}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    
    df =\
    df.append(
        pd.Series([
            data['symbol'],
            data['latestPrice'],
            data['marketCap'],
            np.nan
        ],
        index=columns_
        ),
        ignore_index=True
    )

In [10]:
df

,ticker,stock_price,market_cap,no_of_shares_tobuy
0,A,161.88,48354410684,NaN
1,AAL,21.98,14023210425,NaN
2,AAP,218.99,14030586113,NaN
3,AAPL,148.26,2467102408347,NaN
4,ABBV,116.10,203309257965,NaN
...,...,...,...,...
500,YUM,134.14,40450771591,NaN
501,ZBH,147.80,31185039578,NaN
502,ZBRA,591.74,31786500723,NaN
503,ZION,58.25,9201756358,NaN


In [30]:
# batch execution can be done as well to reduce time required for data fetch
def batch(li, n):
    # get n sized batches from list
    for i in range(0, len(li), n):
        yield li[i:i+n]
# fetch batch based get api request from iex cloud
# endpoint + batch_req_url

In [14]:
# equal weight pf
pf = float(input('Enter Portfolio size:'))

pos_size = pf/df.shape[0]

df['no_of_shares_tobuy'] = df['stock_price'].apply(lambda x: math.floor(pos_size/x))

Enter Portfolio size: 1000000


In [23]:
print('Residual Amount:', end=' ')
print(1000000 - (df['stock_price'] * df['no_of_shares_tobuy']).sum())

Residual Amount: 48685.820000000065


In [142]:
with pd.ExcelWriter('recommended_trades_eq_weight.xlsx') as writer:
    df.to_excel(writer, sheet_name='Recommended Trades', index=False)


## Quantitative Momentum Strategy

### Momentum Equal Weight

In [26]:
ticker = 'AAPL'
api_url = endpoint + f'/stable/stock/{ticker}/stats?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2458256168467,
 'week52high': 155.24,
 'week52low': 105.25,
 'week52highSplitAdjustOnly': 152,
 'week52lowSplitAdjustOnly': 104.9,
 'week52change': 0.311877022977735,
 'sharesOutstanding': 17141179577,
 'float': 0,
 'avg10Volume': 57219142,
 'avg30Volume': 79888735,
 'day200MovingAvg': 133.64,
 'day50MovingAvg': 144.37,
 'employees': 147362,
 'ttmEPS': 5.2,
 'ttmDividendRate': 0.865614422448981,
 'dividendYield': 0.006059597812966607,
 'nextDividendDate': '',
 'exDividendDate': '2021-08-01',
 'nextEarningsDate': '2021-10-14',
 'peRatio': 29.129429653451155,
 'beta': 1.5125505577898422,
 'maxChangePercent': 55.24198673831531,
 'year5ChangePercent': 4.87469174975177,
 'year2ChangePercent': 1.9816054884549552,
 'year1ChangePercent': 0.3556947749886598,
 'ytdChangePercent': 0.10804061559982413,
 'month6ChangePercent': 0.08292322627268472,
 'month3ChangePercent': 0.16395803909783208,
 'month1ChangePercent': 0.006776397000305219,
 'day30ChangePercen

In [72]:
df_momentum = pd.DataFrame(columns=columns_)

for obj in batch(stocks, 100):
    # create batch
    sym_batch_li = obj['Ticker'].to_list()
    sym_batch = ','.join(sym_batch_li)
    
    # fetch batch data
    batch_endpoint = f'/stock/market/batch?symbols={sym_batch}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    api_url = endpoint + batch_endpoint
    data = requests.get(api_url).json()
    
    # set data
    for symbol in sym_batch_li:
        df_momentum =\
        df_momentum.append(
            pd.Series([
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                np.nan
            ],
            index=columns_
            ),
            ignore_index=True
        )
        
df_momentum.columns = ['ticker', 'stock_price', 'year1ChangePercent', 'no_of_shares_tobuy']
df_momentum = df_momentum.sort_values('year1ChangePercent', ascending=False).head(50)

In [88]:
# equal weight pf
pf = float(input('Enter Portfolio size:'))

pos_size = pf/df_momentum.shape[0]

df_momentum['no_of_shares_tobuy'] = df_momentum['stock_price'].apply(lambda x: math.floor(pos_size/x))
df_momentum.sort_values('no_of_shares_tobuy', ascending=False).head()

Enter Portfolio size: 10000


,ticker,stock_price,year1ChangePercent,no_of_shares_tobuy
111,COTY,8.68,1.06428,23
317,MRO,12.70,1.14722,15
174,F,14.55,0.942832,13
265,KIM,22.70,0.872977,8
452,UA,22.70,1.25145,8


### High Quality Momentum Equal Weight

In [92]:
from scipy import stats

In [ ]:
columns_ = [
    'ticker',
    'stock_price',
    'no_of_shares_tobuy',
    'year1ChangePercent',
    'year1ChangePercent_percentile',
    'month6ChangePercent',
    'month6ChangePercent_percentile',
    'month3ChangePercent',
    'month3ChangePercent_percentile',
    'month1ChangePercent',
    'month1ChangePercent_percentile'
]

hq_momentum_df = pd.DataFrame(columns=columns_)

for obj in batch(stocks, 100):
    # create batch
    sym_batch_li = obj['Ticker'].to_list()
    sym_batch = ','.join(sym_batch_li)
    
    # fetch batch data
    batch_endpoint = f'/stock/market/batch?symbols={sym_batch}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    api_url = endpoint + batch_endpoint
    data = requests.get(api_url).json()
    
    # set data
    for symbol in sym_batch_li:
        hq_momentum_df =\
        hq_momentum_df.append(
            pd.Series([
                symbol,
                data[symbol]['price'],
                np.nan,
                data[symbol]['stats']['year1ChangePercent'],
                np.nan,
                data[symbol]['stats']['month6ChangePercent'],
                np.nan,
                data[symbol]['stats']['month3ChangePercent'],
                np.nan,
                data[symbol]['stats']['month1ChangePercent'],
                np.nan
            ],
            index=columns_
            ),
            ignore_index=True
        )

In [102]:
hq_momentum_df =\
hq_momentum_df.dropna(subset=['year1ChangePercent',
                              'month6ChangePercent',
                              'month3ChangePercent',
                              'month1ChangePercent'])

In [126]:
hq_momentum_df['year1ChangePercent_percentile'] =\
hq_momentum_df['year1ChangePercent'].apply(lambda x: stats.percentileofscore(hq_momentum_df['year1ChangePercent'], x))

hq_momentum_df['month6ChangePercent_percentile'] =\
hq_momentum_df['month6ChangePercent'].apply(lambda x: stats.percentileofscore(hq_momentum_df['month6ChangePercent'], x))

hq_momentum_df['month3ChangePercent_percentile'] =\
hq_momentum_df['month3ChangePercent'].apply(lambda x: stats.percentileofscore(hq_momentum_df['month3ChangePercent'], x))

hq_momentum_df['month1ChangePercent_percentile'] =\
hq_momentum_df['month1ChangePercent'].apply(lambda x: stats.percentileofscore(hq_momentum_df['month1ChangePercent'], x))

<ipython-input-126-1e411c41bf4d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hq_momentum_df['year1ChangePercent_percentile'] =\
<ipython-input-126-1e411c41bf4d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hq_momentum_df['month6ChangePercent_percentile'] =\
<ipython-input-126-1e411c41bf4d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [127]:
percentile_cols =\
['year1ChangePercent_percentile', 'month6ChangePercent_percentile', 'month3ChangePercent_percentile', 'month1ChangePercent_percentile']
hq_momentum_df.loc[:,('percentile_mean')] = hq_momentum_df[percentile_cols].mean(axis=1)

In [129]:
hq_momentum_df = hq_momentum_df.sort_values('percentile_mean', ascending=False).head(50)

In [131]:
# equal weight pf
pf = float(input('Enter Portfolio size:'))

pos_size = pf/hq_momentum_df.shape[0]

hq_momentum_df['no_of_shares_tobuy'] = hq_momentum_df['stock_price'].apply(lambda x: math.floor(pos_size/x))
hq_momentum_df.sort_values('no_of_shares_tobuy', ascending=False).head()

Enter Portfolio size: 10000


,ticker,stock_price,no_of_shares_tobuy,year1ChangePercent,year1ChangePercent_percentile,month6ChangePercent,month6ChangePercent_percentile,month3ChangePercent,month3ChangePercent_percentile,month1ChangePercent,month1ChangePercent_percentile,percentile_mean
452,UA,22.20,9,1.27648,96.806387,0.142016,46.107784,0.143718,81.836327,0.165453,97.804391,80.638723
227,HRB,26.67,7,0.779282,85.429142,0.362501,89.820359,0.0975602,71.457086,0.0659568,77.245509,80.988024
245,IPG,38.20,5,1.04423,93.812375,0.56927,98.802395,0.169251,86.027944,0.136487,96.007984,93.662675
249,IRM,47.90,4,0.706739,83.033932,0.457068,96.207585,0.130989,77.844311,0.0692737,79.241517,84.081836
149,DXC,42.17,4,1.10598,95.409182,0.630497,99.201597,0.221752,93.812375,0.0397945,61.477046,87.475050


In [134]:
print('Residual Amount:', end=' ')
print(10000 - (hq_momentum_df['stock_price'] * hq_momentum_df['no_of_shares_tobuy']).sum())

Residual Amount: 5090.319999999999


In [143]:
with pd.ExcelWriter('recommended_trades_hq_eq_weight.xlsx') as writer:
    hq_momentum_df.to_excel(writer, sheet_name='High Quality Stocks', index=False)


## Quantitative Value Strategy